In [17]:
import urllib
import feedparser
import json
import time
from lxml import etree
import xml.etree.ElementTree as ET
import os

In [2]:
phy_cat = ['astro-ph',
'cond-mat.dis-nn',
'cond-mat.mes-hall',
'cond-mat.mtrl-sci',
'cond-mat.other',
'cond-mat.soft',
'cond-mat.stat-mech',
'cond-mat.str-el',
'cond-mat.supr-con',
'gr-qc',
'hep-ex',
'hep-lat',
'hep-ph',
'hep-th',
'math-ph',
'nucl-ex',
'nucl-th',
'physics.acc-ph',
'physics.ao-ph',
'physics.atom-ph',
'physics.atm-clus',
'physics.bio-ph',
'physics.chem-ph',
'physics.class-ph',
'physics.comp-ph',
'physics.data-an',
'physics.flu-dyn',
'physics.gen-ph',
'physics.geo-ph',
'physics.hist-ph',
'physics.ins-det',
'physics.med-ph',
'physics.optics',
'physics.ed-ph',
'physics.soc-ph',
'physics.plasm-ph',
'physics.pop-ph',
'physics.space-ph',
'quant-ph']

In [3]:
bio_cat = ['q-bio.BM', 'q-bio.CB', 'q-bio.GN', 'q-bio.MN', 'q-bio.NC',
'q-bio.OT', 'q-bio.PE', 'q-bio.QM', 'q-bio.SC', 'q-bio.TO']

In [4]:
stat_cat = ['stat.AP', 'stat.CO', 'stat.ML', 'stat.ME', 'stat.TH']

In [5]:
cs_cat = ['cs.AR', 'cs.AI', 'cs.CL', 'cs.CC', 'cs.CE',
'cs.CG', 'cs.GT', 'cs.CV', 'cs.CY', 'cs.CR', 'cs.DS', 
'cs.DB', 'cs.DL', 'cs.DM', 'cs.DC', 'cs.GL', 'cs.GR',
'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO', 'cs.MS', 
'cs.MA', 'cs.MM', 'cs.NI', 'cs.NE', 'cs.NA', 'cs.OS',
'cs.OH', 'cs.PF', 'cs.PL', 'cs.RO', 'cs.SE', 'cs.SD', 'cs.SC']

In [6]:
math_cat = ['nlin.AO', 'nlin.CG', 'nlin.CD', 'nlin.SI', 'nlin.PS',
'math.AG', 'math.AT', 'math.AP', 'math.CT', 'math.CA', 'math.CO',
'math.AC', 'math.CV', 'math.DG', 'math.DS', 'math.FA', 'math.GM',
'math.GN', 'math.GT', 'math.GR', 'math.HO', 'math.IT', 'math.KT',
'math.LO', 'math.MP', 'math.MG', 'math.NT', 'math.NA', 'math.OA',
'math.OC', 'math.PR', 'math.QA', 'math.RT', 'math.RA', 'math.SP',
'math.ST', 'math.SG']

In [65]:
# input manually which discipline and categories from the above will be scrapped
# input manually the current date
# Done: phy_cat(15.11.19), bio_cat, stat_cat, cs_cat, math_cat (16.11.2019)
disc = 'Maths'
categories = ['nlin.AO', 'nlin.CG', 'nlin.CD', 'nlin.SI', 'nlin.PS',
'math.AG', 'math.AT', 'math.AP', 'math.CT', 'math.CA', 'math.CO',
'math.AC', 'math.CV', 'math.DG', 'math.DS', 'math.FA', 'math.GM',
'math.GN', 'math.GT', 'math.GR', 'math.HO', 'math.IT', 'math.KT',
'math.LO', 'math.MP', 'math.MG', 'math.NT', 'math.NA', 'math.OA',
'math.OC', 'math.PR', 'math.QA', 'math.RT', 'math.RA', 'math.SP',
'math.ST', 'math.SG']
c_date = 20191115

In [66]:
print(len(phy_cat))
print(len(bio_cat))
print(len(stat_cat))
print(len(cs_cat))
print(len(math_cat))

39
10
5
36
37


In [67]:
dates = [19810101, 19820101, 19830101, 19840101, 19850101, \
         19860101, 19870101, 19880101, 19890101, 19900101, \
         19910101, 19920101, 19930101, 19940101, 19950101, \
         19960101, 19970101, 19980101, 19990101, 20000101, \
         20010101, 20020101, 20030101, 20040101, 20050101, \
         20060101, 20070101, 20080101, 20090101, 20100101, \
         20110101, 20120101, 20130101, 20140101, 20150101, \
         20160101, 20170101, 20180101, 20190101, 20200101]

In [68]:
# Approach with lxml and feedparser
# Base api query url
# to download papers, pay attention for which categories the script is runnig
base_url = 'http://export.arxiv.org/api/query?';

for cat in categories:
    
    if not os.path.exists('dataset/'+c_date+'/'+disc+'/'+cat):
        os.makedirs('dataset/'+c_date+'/'+disc+'/'+cat)  
    
    i = 0
    while i != len(dates)-1:
        print(i, dates[i])
    #for i in range(len(dates)-1):
        error = 0                    
        start = 0                    
        max_results = 2000
        t = 0
        query_results = 0
        data = []

        while error == False:

            print(t)
            query_cat = 'search_query=cat:%s+AND+' %(cat)
            query_time = 'submittedDate:[%i*+TO+%i*]&' %(dates[i], dates[i+1])
            query_range = 'start=%i&max_results=%i&' %(start,max_results)
            query_sort = 'sortBy=submittedDate&sortOrder=descending' 

            # Opensearch metadata such as totalResults, startIndex, 
            # and itemsPerPage live in the opensearch namespase.
            # Some entry metadata lives in the arXiv namespace.
            # This is a hack to expose both of these namespaces in
            # feedparser v4.1
            feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
            feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'
            
            # using feedparser
            # perform a GET request using the base_url and query
            response = urllib.request.urlopen(base_url+query_cat+query_time+query_range+query_sort).read()
            # parse the response using feedparser
            feed = feedparser.parse(response)

            # using lxml
            # creating parser
            parser = etree.HTMLParser()
            # opneing url
            with urllib.request.urlopen(base_url+query_cat+query_time+query_range+query_sort) as f:
                tree = ET.parse(f, parser)
            # defining root
            root = tree.getroot()
            
            # feedparser part
            # print out feed information
            print('Feed title: %s' % feed.feed.title)
            print('Feed last updated: %s' % feed.feed.updated)

            # print opensearch metadata
            print('totalResults for this query: %s' % feed.feed.opensearch_totalresults)
            print('itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
            print('startIndex for this query: %s'   % feed.feed.opensearch_startindex)

            query_results = int(feed.feed.opensearch_totalresults)

            if int(query_results) == 0:
                #os.rmdir(cat+'/'+str(int(str(dates[i])[:4])))
                i += 1
                break
            else:
                if len(feed.entries) == 0:
                    time.sleep(5)
                    continue

            # lmxl part
            entries = [entry for entry in root.iter('entry')]
            # Run through each entry, and print out information
            print('Entries detected (feedparser): ', len(feed.entries))
            print('Entries detected (lxml): ', len(entries))
            if len(entries) == min(query_results-start,2000):
                pass
            else:
                continue
            
            for entry in entries:
                item = {'arxiv-id':entry.find('id').text.split('/abs/')[-1]}
                item['Published'] = entry.find('published').text
                item['Title'] = entry.find('title').text
                authors = [author for author in entry.iter('author')]
                item['Authors'] = []
                for author in authors:
                    author_info = {}
                    try:
                        author_info['Name'] = author.find('name').text
                    except:
                        author_info['Name'] = 'unknown'
                    try:
                        author_info['Affiliation'] = author.find('affiliation').text
                    except:
                        author_info['Affiliation'] = 'unknown'
                    item['Authors'].append(author_info)
                # The abstract is in the <summary> element
                item['Abstract'] = entry.find('summary').text
                
                data.append(item)


            print('Entries downloaded',len(data))
            print(' ')
            time.sleep(5)

            start += max_results   
            t += 1
            if t > int(feed.feed.opensearch_totalresults.encode('ascii','ignore'))/2000:
                error = 1
                
        if len(data) != 0:
            jsonData=json.dumps(data)
            f = open('dataset/'+c_date+'/'+disc+'/'+cat+'/'+str(int(str(dates[i])))+'.json', 'w')
            f.write(jsonData)
            f.close()
            i += 1

0 19810101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[19810101* TO 19820101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
1 19820101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[19820101* TO 19830101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
2 19830101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[19830101* TO 19840101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
3 19840101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[19840101* TO 19850101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019

Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[20060101* TO 20070101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 142
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  142
Entries detected (lxml):  142
Entries downloaded 142
 
26 20070101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[20070101* TO 20080101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 131
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  131
Entries detected (lxml):  131
Entries downloaded 131
 
27 20080101
0
Feed title: ArXiv Query: search_query=cat:nlin.AO AND submittedDate:[20080101* TO 20090101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 155
itemsPerPage for this query: 2000
startIndex for 

Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[19900101* TO 19910101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
10 19910101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[19910101* TO 19920101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
11 19920101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[19920101* TO 19930101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
12 19930101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[19930101* TO 19940101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:

32 20130101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[20130101* TO 20140101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 63
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  63
Entries detected (lxml):  63
Entries downloaded 63
 
33 20140101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[20140101* TO 20150101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 49
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  49
Entries detected (lxml):  49
Entries downloaded 49
 
34 20150101
0
Feed title: ArXiv Query: search_query=cat:nlin.CG AND submittedDate:[20150101* TO 20160101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 69
itemsPerPage for this query: 2000
startIndex

18 19990101
0
Feed title: ArXiv Query: search_query=cat:nlin.CD AND submittedDate:[19990101* TO 20000101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 596
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  596
Entries detected (lxml):  596
Entries downloaded 596
 
19 20000101
0
Feed title: ArXiv Query: search_query=cat:nlin.CD AND submittedDate:[20000101* TO 20010101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 497
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  497
Entries detected (lxml):  497
Entries downloaded 497
 
20 20010101
0
Feed title: ArXiv Query: search_query=cat:nlin.CD AND submittedDate:[20010101* TO 20020101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 517
itemsPerPage for this query: 2000
s

...

Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[19910101* TO 19920101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 0
itemsPerPage for this query: 2000
startIndex for this query: 0
11 19920101
0
Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[19920101* TO 19930101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 2
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  2
Entries detected (lxml):  2
Entries downloaded 2
 
12 19930101
0
Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[19930101* TO 19940101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 3
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  3
Entries detected (lxml):  3
Entries downloaded 3
 
13 1

Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[20130101* TO 20140101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 446
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  446
Entries detected (lxml):  446
Entries downloaded 446
 
33 20140101
0
Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[20140101* TO 20150101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 503
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  100
Entries detected (lxml):  503
Entries downloaded 503
 
34 20150101
0
Feed title: ArXiv Query: search_query=cat:math.SP AND submittedDate:[20150101* TO 20160101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 561
itemsPerPage for this query: 2000
startIndex for 

18 19990101
0
Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[19990101* TO 20000101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 2
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  2
Entries detected (lxml):  2
Entries downloaded 2
 
19 20000101
0
Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[20000101* TO 20010101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 9
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  9
Entries detected (lxml):  9
Entries downloaded 9
 
20 20010101
0
Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[20010101* TO 20020101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 17
itemsPerPage for this query: 2000
startIndex for thi

Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[20160101* TO 20170101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 1203
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  1203
Entries detected (lxml):  1203
Entries downloaded 1203
 
36 20170101
0
Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[20170101* TO 20180101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 1268
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  1268
Entries detected (lxml):  1268
Entries downloaded 1268
 
37 20180101
0
Feed title: ArXiv Query: search_query=cat:math.ST AND submittedDate:[20180101* TO 20190101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 1308
itemsPerPage for this query: 2000
startI

19 20000101
0
Feed title: ArXiv Query: search_query=cat:math.SG AND submittedDate:[20000101* TO 20010101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 179
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  179
Entries detected (lxml):  179
Entries downloaded 179
 
20 20010101
0
Feed title: ArXiv Query: search_query=cat:math.SG AND submittedDate:[20010101* TO 20020101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 174
itemsPerPage for this query: 2000
startIndex for this query: 0
Entries detected (feedparser):  174
Entries detected (lxml):  174
Entries downloaded 174
 
21 20020101
0
Feed title: ArXiv Query: search_query=cat:math.SG AND submittedDate:[20020101* TO 20030101*]&id_list=&start=0&max_results=2000
Feed last updated: 2019-11-16T00:00:00-05:00
totalResults for this query: 226
itemsPerPage for this query: 2000
s